In [1]:
from explanation.mojito2 import getMixedTriangles,createPerturbationsFromTriangle
from deeper.deeper_utils import dataset_tolist

Using TensorFlow backend.


In [2]:
import pandas as pd
data = pd.read_csv('datasets/itunes_amazon/augmented_test.csv')
itunes = pd.read_csv('datasets/itunes_amazon/itunes.csv',dtype=str)
amazon = pd.read_csv('datasets/itunes_amazon/amazon.csv',dtype=str)
allTriangles = getMixedTriangles(data,[itunes,amazon])

In [6]:
len(allTriangles)

82

In [7]:
attributes = [col for col in itunes.columns if col not in ['id']]

In [19]:
perturbations,pertattr = createPerturbationsFromTriangle(allTriangles[1],attributes,1,1)
perturbations['changedAttribute'] = pertattr

In [20]:
perturbations.head(8)

,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released,changedAttribute
0,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Where Did It All Go Wrong ?,Sam Hunt,Montevallo,"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:46,27-Oct-14,"(Song_Name,)"
1,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,U2,Montevallo,"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:46,27-Oct-14,"(Artist_Name,)"
2,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,Sam Hunt,Achtung Baby ( Deluxe Edition ),"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:46,27-Oct-14,"(Album_Name,)"
3,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,Sam Hunt,Montevallo,Rock,$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:46,27-Oct-14,"(Genre,)"
4,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,Sam Hunt,Montevallo,"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:46,27-Oct-14,"(Price,)"
5,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,Sam Hunt,Montevallo,"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,( C ) 2011 Universal-Island Records Limited un...,3:46,27-Oct-14,"(CopyRight,)"
6,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,Sam Hunt,Montevallo,"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:56,27-Oct-14,"(Time,)"
7,Speakers,Sam Hunt,Montevallo,Country,$ 1.29,"( C ) 2014 MCA Nashville , a Division of UMG R...",3:48,"October 27 , 2014",Speakers,Sam Hunt,Montevallo,"Country , Music , Pop , Pop/Rock , Honky Tonk ...",$ 1.29,"2014 MCA Nashville , a Division of UMG Recordi...",3:46,"October 31 , 2011","(Released,)"


In [16]:
datalist = dataset_tolist(perturbations,'ltable_','rtable_',ignore_columns=['changedAttribute'])

In [18]:
datalist[0]

(['Speakers',
  'Sam',
  'Hunt',
  'Montevallo',
  'Country',
  '$',
  '1.29',
  '(',
  'C',
  ')',
  '2014',
  'MCA',
  'Nashville',
  ',',
  'a',
  'Division',
  'of',
  'UMG',
  'Recordings',
  ',',
  'Inc.',
  '3:48',
  'October',
  '27',
  ',',
  '2014'],
 ['Speakers',
  'U2',
  'Achtung',
  'Baby',
  '(',
  'Deluxe',
  'Edition',
  ')',
  'Rock',
  '$',
  '1.29',
  '(',
  'C',
  ')',
  '2011',
  'Universal-Island',
  'Records',
  'Limited',
  'under',
  'exclusive',
  'licence',
  'to',
  'Mercury',
  'Records',
  'Limited',
  'in',
  'the',
  'UK',
  ',',
  'Interscope',
  'Records',
  'in',
  'the',
  'US',
  'and',
  'Universal',
  'Music',
  'Group',
  'for',
  'the',
  'rest',
  'of',
  'the',
  'world',
  '3:56',
  'October',
  '31',
  ',',
  '2011'])

In [3]:
augmented_test = pd.read_csv('datasets/DBLP-Scholar/augmented_test.csv')
test = pd.read_csv('datasets/DBLP-Scholar/test.csv')

In [5]:
newsamples = augmented_test.loc[2150:2880]
newsamples.head()

,label,ltable_title,ltable_authors,ltable_venue,ltable_year,rtable_title,rtable_authors,rtable_venue,rtable_year,id
2150,0,Transactional Client-Server Cache Consistency:...,"M Franklin, M Carey, M Livny",ACM Trans. Database Syst.,1997,ACDN: a content delivery network for applicati...,"P Karbhari, M Rabinovich, Z Xiao, F Douglis","Proceedings of ACM SIGMOD,",NaN,journals/tods/FranklinCL97#Mw3xLuyR3qQJ
2151,0,APEX: an adaptive path index for XML data,"C Chung, J Min, K Shim",SIGMOD Conference,2002,Programming and control of kinematically redun...,"J Baillieul, JM Hollerbach, RW Brockett",Proceedings of the 23rd IEEE conference on dec...,NaN,conf/sigmod/ChungMS02#aIbNhvHkBP8J
2152,0,APEX: an adaptive path index for XML data,"C Chung, J Min, K Shim",SIGMOD Conference,2002,Implementing Data Cubes Efficiently,"VHA Rajaraman, JD Ullman","Proc. ACM SIGMOD,",1996.0,conf/sigmod/ChungMS02#-RzBfKSBxpgJ
2153,0,APEX: an adaptive path index for XML data,"C Chung, J Min, K Shim",SIGMOD Conference,2002,Improving Logging and Recovery Performance in ...,"R Barga, S Chen, D Lomet","Data Engineering, 2004. Proceedings. 20th Inte...",NaN,conf/sigmod/ChungMS02#UjcuBBQi2FcJ
2154,0,Monitoring Business Processes through Event Co...,"A Adi, D Botzer, O Etzion, T Yatzkar-Haham",SIGMOD Conference,2001,Regulation of UCP1 and UCP2 gene expression by...,"CP Portocarrero, DE Bauman, DM Barbano, JR Zie...","San Diego,",NaN,conf/sigmod/AdiBEY01#XOzaLnX6k84J


In [6]:
newsamples_frac = newsamples.sample(frac=0.3)

In [7]:
augmented_test2 = pd.concat([test,newsamples_frac],ignore_index=True)
augmented_test2.to_csv('datasets/DBLP-Scholar/augmented_test2.csv',index=False)
len(augmented_test2)

2358

## Test deepmatcher on itunes amazon

In [1]:
import deepmatcher as dm
import pandas as pd
from explanation.deepmatcher_utils import wrapDm

In [3]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('models/itunes_amazon_hybrid.pth')

In [5]:
test = pd.read_csv('datasets/itunes_amazon/augmented_test.csv')
testneg = test[test.label==0]
len(testneg)

91

In [6]:
import numpy as np
prediction = wrapDm(testneg,model)
np.count_nonzero(np.argmax(prediction,axis=1)==0)

90

In [7]:
testneg_c = testneg.copy()
testneg_c['ltable_Time'] = testneg_c['rtable_Time']
np.count_nonzero(np.argmax(wrapDm(testneg_c,model),axis=1)==0)

85

In [8]:
rnn_model = dm.MatchingModel(attr_summarizer='rnn')
rnn_model.load_state('models/itunes_amazon_rnn.pth')

In [11]:
testneg_c = testneg.copy()
testneg_c['ltable_Time'] = testneg_c['rtable_Time']
np.count_nonzero(np.argmax(wrapDm(testneg_c,rnn_model),axis=1)==0)

64